## Library imports

In [1]:
import os, sys
import numpy as np

import torch
import torch.nn as nn

## Helper functions

In [2]:
def getProjectRootPath() -> str:
    projectRootPath  = os.path.dirname(sys.path[0])
    return projectRootPath

projectRootPath = getProjectRootPath()

In [3]:
def exportModel(model : nn.Module, sampleInput : torch.Tensor, outputPath : str):
    model.eval();
    torch.onnx.export(model,               # model being run
                  sampleInput,                         # model input (or a tuple for multiple inputs)
                  outputPath,   # where to save the model (can be a file or file-like object)
                  export_params=True,        # store the trained parameter weights inside the model file
                  opset_version=10,          # the ONNX version to export the model to
                  do_constant_folding=True,  # whether to execute constant folding for optimization
                  input_names = ['Model Input'],   # the model's input names
                  output_names = ['Model Output'], # the model's output names
                  dynamic_axes={'input' : {0 : 'batch_size'},    # variable length axes
                                'output' : {0 : 'batch_size'}})
    return

## File constants

In [4]:
# saanple tensor for model input
sampleInput = torch.Tensor(1, 3, 224, 224)

# file path constants
dataFolder = os.path.join(projectRootPath, *['data'])
modelOutputPath = os.path.join(dataFolder, *['FCN_out.onnx'])

In [5]:
from torchvision.models.segmentation import fcn_resnet50

In [6]:
model = fcn_resnet50()

In [7]:
exportModel(model, sampleInput, outputPath=modelOutputPath)

D:\surya\miniconda37\envs\imgseg\lib\site-packages\torch\onnx\utils.py:1294: UserWarning: Provided key input for dynamic axes is not a valid input/output name
  warnings.warn("Provided key {} for dynamic axes is not a valid input/output name".format(key))
D:\surya\miniconda37\envs\imgseg\lib\site-packages\torch\onnx\utils.py:1294: UserWarning: Provided key output for dynamic axes is not a valid input/output name
  warnings.warn("Provided key {} for dynamic axes is not a valid input/output name".format(key))
D:\surya\miniconda37\envs\imgseg\lib\site-packages\torch\onnx\symbolic_helper.py:382: UserWarning: You are trying to export the model with onnx:Resize for ONNX opset version 10. This operator might cause results to not match the expected results by PyTorch.
ONNX's Upsample/Resize operator did not match Pytorch's Interpolation until opset 11. Attributes to determine how to transform the input were added in onnx:Resize in opset 11 to support Pytorch's behavior (like coordinate_transfo

In [9]:
for name,_ in model.named_children():
    print(name)

backbone
classifier


In [11]:
model.classifier

FCNHead(
  (0): Conv2d(2048, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (1): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU()
  (3): Dropout(p=0.1, inplace=False)
  (4): Conv2d(512, 21, kernel_size=(1, 1), stride=(1, 1))
)

In [12]:
backboneOutput = model.backbone(sampleInput)

In [27]:
exportModel(model.classifier, backboneOutput['out'], os.path.join(dataFolder, *['FCNHead.onnx']))